# RSF - VIMP

In [47]:
################################################################################
# SET UP
################################################################################

import os
import sys
import time
import numpy as np
import pandas as pd
import joblib
import pickle
from collections import Counter
#from sksurv.util import Surv
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import OneHotEncoder

# Set working directory
os.chdir(os.path.expanduser("~/PhD_Workspace/PredictRecurrence/"))

# Import custom RSF functions
sys.path.append("/Users/le7524ho/PhD_Workspace/PredictRecurrence/src/")
from src.utils import (
    log,
    load_training_data,
    beta2m,
    subset_methylation)
from src.annotation_functions import (
    run_univariate_cox_for_cpgs
)

In [48]:
################################################################################
# INPUT FILES
################################################################################

infile_map = {
    "ERpHER2n_Clinical" : "./output/RSF/ERpHER2n/Clinical/None/outer_cv_models.pkl", 
    "ERpHER2n_Combined" : "./output/RSF/ERpHER2n/Combined/Unadjusted/outer_cv_models.pkl",
    "ERpHER2n_Methylation" : "./output/RSF/ERpHER2n/Methylation/Unadjusted/outer_cv_models.pkl",

    "TNBC_Clinical" : "./output/RSF/TNBC/Clinical/None/outer_cv_models.pkl",
    "TNBC_Combined" : "./output/RSF/TNBC/Combined/Unadjusted/outer_cv_models.pkl", 
    "TNBC_Methylation" : "./output/RSF/TNBC/Methylation/Unadjusted/outer_cv_models.pkl", 

    "All_Clinical" : "./output/RSF/All/Clinical/None/outer_cv_models.pkl", 
    "All_Combined" : "./output/RSF/All/Combined/Unadjusted/outer_cv_models.pkl", 
    "All_Methylation" : "./output/RSF/All/Methylation/Unadjusted/outer_cv_models.pkl" 
}

In [49]:

################################################################################
# PARAMS
################################################################################

# Output directory and files
output_dir = "output/RSF/VIMP_analysis/" # ⚠️ ADAPT
os.makedirs(output_dir, exist_ok=True)
#outfile_univariate_cox = os.path.join(output_dir, "testset_univariate_cox.csv")


In [50]:
outerfolds = {}
for key, filepath in infile_map.items():
    outerfolds[key] = joblib.load(filepath)

## Test example: ERpHER2n_Combined models

In [21]:
ERpHER2n_Combined_OuterDicts = outerfolds['ERpHER2n_Combined']

In [51]:
# Input files
infile_betavalues = "./data/train/train_methylation_unadjusted.csv" # ⚠️ ADAPT
infile_clinical = "./data/train/train_clinical.csv"
infile_train_ids = "./data/train/train_subcohorts/ERpHER2n_train_ids.csv" # sample ids of training cohort

In [52]:
# Load and prepare data
train_ids = pd.read_csv(infile_train_ids, header=None).iloc[:, 0].tolist()
beta_matrix, clinical_data = load_training_data(train_ids, infile_betavalues, infile_clinical)

# convert to M-values
mvals = beta2m(beta_matrix, beta_threshold=0.001)
infile_cpg_ids = "./data/set_definitions/CpG_prefiltered_sets/cpg_ids_atac_overlap.txt"

# admin censoring for tnbc
# subset methylation atac overlap (needed to match ids)
mvals = subset_methylation(mvals,infile_cpg_ids)
X = mvals.copy()

Loaded training data.
Successfully loaded 205799 CpG IDs for pre-filtering.
Successfully subsetted methylation data to 193246 pre-filtered CpGs.


In [53]:
# onehot encode cat clinvars
# subset clinical data aligned to X
clin = clinical_data[["Age", "Size.mm", "NHG", "LN"]].loc[X.index]
# one-hot encode the categorical clinical variables
encoder = OneHotEncoder(drop=None, dtype=float, sparse_output=False)
encoded = encoder.fit_transform(clin[["NHG", "LN"]])

encoded_cols = encoder.get_feature_names_out(["NHG", "LN"]).tolist()

# make a DataFrame for the encoded columns
encoded_df = pd.DataFrame(encoded, columns=encoded_cols, index=X.index)
# build the encoded clinical DataFrame (drop original categorical cols)
clin_encoded = pd.concat([clin.drop(columns=["NHG", "LN"]), encoded_df], axis=1)
# concatenate encoded clinical back into X
X = pd.concat([X, clin_encoded], axis=1).copy()

# build clinvars_included_encoded: replace original categorical names with encoded column names
clinvars_included_encoded = [c for c in ["Age", "Size.mm", "NHG", "LN"] if c not in ["NHG", "LN"]] + encoded_cols
log(f"Added {clinvars_included_encoded} clinical variables. New X shape: {X.shape}")


=== Added ['Age', 'Size.mm', 'NHG_1', 'NHG_2', 'NHG_3', 'LN_N+', 'LN_N0'] clinical variables. New X shape: (1008, 193253) ===



In [ ]:
print(f"Best cv fold: {selected_fold['fold']}")
selected_model = selected_fold['model']
bm_testidx = selected_fold["test_idx"].tolist() 
bm_trainidx = selected_fold["train_idx"].tolist()
bm_testids = selected_fold["test_ids"].tolist() 
bm_trainids = selected_fold["train_ids"].tolist()
input_training_features = selected_fold["input_training_features"].tolist()

selected_fold["model"]


In [23]:
ERpHER2n_Combined_OuterDicts[1]

{'fold': 1,
 'model': Pipeline(steps=[('columntransformer',
                  ColumnTransformer(transformers=[('scale', StandardScaler(),
                                                   ['Age', 'Size.mm',
                                                    'cg07404514', 'cg13484549',
                                                    'cg09084370', 'cg08111414',
                                                    'cg05995697', 'cg00366818',
                                                    'cg04861380', 'cg00421363',
                                                    'cg15297153', 'cg09504482',
                                                    'cg25908973', 'cg04651042',
                                                    'cg09272992', 'cg02251557',
                                                    'cg13077472', 'cg17935217',
                                                    'cg10248492', 'cg03654617',
                                                    'cg2...
            

In [54]:
from sksurv.util import Surv

y = Surv.from_dataframe("RFi_event", "RFi_years", clinical_data)

In [55]:
# Quick test on one fold
fold_0 = ERpHER2n_Combined_OuterDicts[0]
features = fold_0.get("features_after_filter2") or fold_0.get("features_after_filter1")
X_test = X.iloc[fold_0["test_idx"]][features]
y_test = y[fold_0["test_idx"]]

# Test different tree counts
from sksurv.ensemble import RandomSurvivalForest

for n_trees in [400, 800, 1000, 1500]:
    # Get best params
    best_params = {
        k.replace("estimator__randomsurvivalforest__", ""): v 
        for k, v in fold_0["cv_results"]["params"][fold_0["cv_results"]["rank_test_score"].argmin()].items()
        if k.startswith("estimator__")
    }
    best_params['n_estimators'] = n_trees
    best_params['n_jobs'] = -1
    
    # Train and test
    rsf = RandomSurvivalForest(**best_params)
    X_train = X.iloc[fold_0["train_idx"]][features]
    y_train = y[fold_0["train_idx"]]
    
    # Transform with preprocessing
    X_train_trans = fold_0["model"].named_steps[list(fold_0["model"].named_steps.keys())[0]].transform(X_train)
    X_test_trans = fold_0["model"].named_steps[list(fold_0["model"].named_steps.keys())[0]].transform(X_test)
    
    rsf.fit(X_train_trans, y_train)
    score = rsf.score(X_test_trans, y_test)
    
    print(f"{n_trees} trees: C-index = {score:.4f}")

400 trees: C-index = 0.6398
800 trees: C-index = 0.6186
1000 trees: C-index = 0.6294
1500 trees: C-index = 0.6304


In [56]:
# Check actual feature usage in your current models
fold_0 = ERpHER2n_Combined_OuterDicts[0]
rsf = fold_0["model"].named_steps['randomsurvivalforest']

n_trees = rsf.n_estimators
n_features_total = len(fold_0.get("features_after_filter2") or fold_0.get("features_after_filter1"))
max_features = rsf.max_features

if max_features == "sqrt":
    max_features_per_tree = int(np.sqrt(n_features_total))
elif isinstance(max_features, float):
    max_features_per_tree = int(max_features * n_features_total)
else:
    max_features_per_tree = max_features

expected_appearances = n_trees * (max_features_per_tree / n_features_total)

print(f"Total features: {n_features_total}")
print(f"Features per tree: {max_features_per_tree}")
print(f"Number of trees: {n_trees}")
print(f"Expected appearances per feature: {expected_appearances:.1f}")

if expected_appearances < 3:
    print("⚠️ Warning: Features appear too rarely. Consider more trees.")
elif expected_appearances > 20:
    print("ℹ️ Note: Features appear very frequently. Could use fewer trees.")
else:
    print("✅ Good: Adequate tree count for feature coverage.")

Total features: 5007
Features per tree: 25
Number of trees: 1500
Expected appearances per feature: 7.5
✅ Good: Adequate tree count for feature coverage.


In [37]:
from scipy.stats import spearmanr
import pandas as pd
import numpy as np

results = {}

for entry in ERpHER2n_Combined_OuterDicts:
    fold = entry["fold"]
    if entry["model"] is None:
        continue
    
    model = entry["model"]
    test_idx = entry["test_idx"]
    train_idx = entry["train_idx"]
    
    features_to_use = entry.get("features_after_filter2") or entry.get("features_after_filter1")
    X_test = X.iloc[test_idx][features_to_use]
    y_train, y_test = y[train_idx], y[test_idx]
    
    print(f"\nFold {fold}: {len(features_to_use)} features", flush=True)
    
    # Baseline
    baseline_score = model.score(X_test, y_test)
    pred_scores = model.predict(X_test)
    
    # Screen by correlation
    correlations = []
    for col in X_test.columns:
        corr, _ = spearmanr(X_test[col], pred_scores)
        correlations.append({'feature': col, 'correlation': abs(corr)})
    
    corr_df = pd.DataFrame(correlations).sort_values('correlation', ascending=False)
    top_50 = corr_df.head(50)['feature'].tolist()
    
    # Permutation importance
    print(f"  Permutation test on top 50...", flush=True)
    perm_results = []
    
    for i, feature in enumerate(top_50):
        if (i + 1) % 10 == 0:
            print(f"    {i+1}/50 features tested", flush=True)
        
        scores = []
        for _ in range(10):
            X_perm = X_test.copy()
            X_perm[feature] = np.random.RandomState(42).permutation(X_perm[feature].values)
            scores.append(baseline_score - model.score(X_perm, y_test))
        
        perm_results.append({
            'feature': feature,
            'importance_mean': np.mean(scores),
            'importance_std': np.std(scores)
        })
    
    results[fold] = {
        'correlations': corr_df,
        'permutation': pd.DataFrame(perm_results).sort_values('importance_mean', ascending=False),
        'baseline': baseline_score
    }
    
    print(f"  ✓ Fold {fold} done", flush=True)


Fold 0: 5007 features
  Permutation test on top 50...
    10/50 features tested
    20/50 features tested
    30/50 features tested
    40/50 features tested
    50/50 features tested
  ✓ Fold 0 done

Fold 1: 5007 features
  Permutation test on top 50...
    10/50 features tested
    20/50 features tested
    30/50 features tested
    40/50 features tested
    50/50 features tested
  ✓ Fold 1 done


In [46]:
import pandas as pd
import numpy as np

# --- 1. AGGREGATE PERMUTATION IMPORTANCE (VIMP) ---

print("\n--- 1. Aggregating Cross-Validated Permutation Importance (VIMP) ---\n")

all_vimp_dfs = []

# Collect VIMP data from all folds
for fold, data in results.items():
    # The 'permutation' key holds a DataFrame with columns: 
    # ['feature', 'importance_mean', 'importance_std']
    vimp_df = data['permutation'].copy()
    vimp_df['fold'] = fold
    all_vimp_dfs.append(vimp_df)

# Combine all VIMP results
combined_vimp = pd.concat(all_vimp_dfs, ignore_index=True)

# Calculate Mean and Standard Deviation across all folds
vimp_overview = combined_vimp.groupby('feature').agg(
    mean_vimp=('importance_mean', 'mean'),
    std_vimp=('importance_mean', 'std'),
    # Count how many folds the feature appeared in the top 50
    fold_count=('fold', 'count')
).reset_index()

# Filter, Sort, and Display Top 20 VIMP Features
vimp_overview = vimp_overview[vimp_overview['mean_vimp'] > 0]
vimp_overview = vimp_overview.sort_values(by='mean_vimp', ascending=False)

print("## Top 20 Cross-Validated Permutation Importances")
print(vimp_overview.head(20).to_markdown(index=False, floatfmt=".5f"))

# --------------------------------------------------------------------------------

# --- 2. MODEL PERFORMANCE STABILITY (C-INDEX) ---

print("\n--- 2. Model Performance Stability ---\n")

# Collect all baseline scores (C-index from the test set)
baseline_scores = [data['baseline'] for data in results.values()]

mean_score = np.mean(baseline_scores)
std_score = np.std(baseline_scores)

# Create a DataFrame for easy display
performance_stability_df = pd.DataFrame({
    'Metric': ['Baseline C-index (Test Set)'],
    'Mean Score': [mean_score],
    'Std Dev': [std_score]
})

print("## Performance Stability Across Outer Folds")
print(performance_stability_df.to_markdown(index=False, floatfmt=".4f"))

# --------------------------------------------------------------------------------

# --- 3. FEATURE SELECTION CONSISTENCY (SPEARMAN TOP 50) ---

print("\n--- 3. Feature Selection Consistency ---\n")

top_features_per_fold = []
num_folds = len(results)

# Collect all top 50 features selected by Spearman correlation across all folds
for data in results.values():
    top_features_per_fold.extend(data['correlations'].head(50)['feature'].tolist())

# Count frequency
freq_series = pd.Series(top_features_per_fold).value_counts()
freq_df = freq_series.to_frame('Frequency').reset_index().rename(columns={'index': 'Feature'})

# Calculate percentage of folds it appeared in
freq_df['Frequency (%)'] = (freq_df['Frequency'] / num_folds * 100).round(1)

# Sort and Display Top 10 Most Consistent Features
freq_df = freq_df.sort_values('Frequency', ascending=False)

print("## Top 10 Feature Selection Consistency (Spearman Top 50 Screen)")
print(freq_df.head(10).to_markdown(index=False))


--- 1. Aggregating Cross-Validated Permutation Importance (VIMP) ---

## Top 20 Cross-Validated Permutation Importances
| feature    |   mean_vimp |   std_vimp |   fold_count |
|:-----------|------------:|-----------:|-------------:|
| cg23167906 |     0.00057 |  nan       |            1 |
| cg09155701 |     0.00056 |  nan       |            1 |
| cg21347053 |     0.00042 |  nan       |            1 |
| cg11986813 |     0.00030 |    0.00007 |            2 |
| cg04744755 |     0.00028 |    0.00040 |            2 |
| cg16707405 |     0.00019 |  nan       |            1 |
| cg03918605 |     0.00019 |  nan       |            1 |
| cg24366702 |     0.00019 |  nan       |            1 |
| cg09658429 |     0.00014 |  nan       |            1 |
| cg15297153 |     0.00014 |  nan       |            1 |
| cg08513472 |     0.00014 |  nan       |            1 |
| cg26013553 |     0.00014 |  nan       |            1 |
| cg25259804 |     0.00014 |  nan       |            1 |
| cg17019898 |     0.000

In [ ]:
results[1]['']

dict_items([('correlations',          feature  correlation
53    cg01423964     0.522305
301   cg11595545     0.515009
4760  cg08513472     0.513077
264   cg07808555     0.507287
575   cg20302133     0.499534
...          ...          ...
2148  cg08512745     0.000325
2579  cg25715429     0.000164
4343  cg17058782     0.000111
1689  cg06366981     0.000085
3565  cg13588023     0.000034

[5007 rows x 2 columns]), ('permutation',        feature  importance_mean  importance_std
15  cg04744755         0.000561             0.0
44  cg09155701         0.000561             0.0
9   cg21347053         0.000421             0.0
48  cg11986813         0.000351             0.0
8   cg22661116         0.000210             0.0
10  cg15297153         0.000140             0.0
2   cg08513472         0.000140             0.0
6   cg09658429         0.000140             0.0
32  cg26013553         0.000140             0.0
39  cg25259804         0.000140             0.0
3   cg07808555         0.000070         

In [35]:
ERpHER2n_Combined_OuterDicts[0].items()

dict_items([('fold', 0), ('model', Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('scale', StandardScaler(),
                                                  ['Age', 'Size.mm',
                                                   'cg24968291', 'cg13407274',
                                                   'cg20033401', 'cg02541301',
                                                   'cg20649018', 'cg24725146',
                                                   'cg17098449', 'cg26731073',
                                                   'cg05995697', 'cg16019898',
                                                   'cg09471709', 'cg09579704',
                                                   'cg24701662', 'cg03252700',
                                                   'cg10474350', 'cg19819135',
                                                   'cg19115690', 'cg26863308',
                                                   'cg0...
           

In [30]:
results = {}
for entry in ERpHER2n_Combined_OuterDicts:
    fold = entry["fold"]

    if entry["model"] is None:
        print(f"  Skipping fold {fold} (no model)", flush=True)
        continue

    model = entry["model"]
    test_idx = entry["test_idx"]
    train_idx = entry["train_idx"]

    model_name = model.named_steps[list(model.named_steps.keys())[-1]].__class__.__name__
    print(f"Evaluating fold {fold} ({model_name})...", flush=True)

    # Subset data
    # Use filter2 if available, otherwise filter1, otherwise all columns
    # Use the most refined feature set available
    if entry.get("features_after_filter2") is not None:
        features_to_use = entry["features_after_filter2"]
    elif entry.get("features_after_filter1") is not None:
        features_to_use = entry["features_after_filter1"]
    else:
        # This should never happen if training function works correctly
        raise ValueError(f"Fold {fold}: No feature list found in entry. "
                    "Check that training function stores 'features_after_filter1'.")

    X_test = X.iloc[test_idx][features_to_use]
    y_train, y_test = y[train_idx], y[test_idx]

    res = permutation_importance(model, X_test, y_test, n_repeats=2, random_state=42, n_jobs=-1)
    results[fold] = res

0
Evaluating fold 0 (RandomSurvivalForest)...


/Users/le7524ho/miniconda3/envs/predict_recurrence/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


KeyboardInterrupt: 

In [11]:
outerfolds['ERpHER2n_Combined'][0]['model']

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('scale', StandardScaler(),
                                                  ['Age', 'Size.mm',
                                                   'cg24968291', 'cg13407274',
                                                   'cg20033401', 'cg02541301',
                                                   'cg20649018', 'cg24725146',
                                                   'cg17098449', 'cg26731073',
                                                   'cg05995697', 'cg16019898',
                                                   'cg09471709', 'cg09579704',
                                                   'cg24701662', 'cg03252700',
                                                   'cg10474350', 'cg19819135',
                                                   'cg19115690', 'cg26863308',
                                                   'cg0...
                                                   'cg09234966', 'cg23446931',
                                                   'cg25977879', 'cg18019042',
                                                   'cg03317517', 'cg07547418',
                                                   'cg08176140', 'cg23092148', ...]),
                                                 ('passthrough_encoded',
                                                  'passthrough',
                                                  ['NHG_1', 'NHG_2', 'NHG_3',
                                                   'LN_N+', 'LN_N0'])],
                                   verbose_feature_names_out=False)),
                ('randomsurvivalforest',
                 RandomSurvivalForest(max_depth=10, max_features=0.005,
                                      min_samples_leaf=50, min_samples_split=60,
                                      n_estimators=1500, n_jobs=-1))])

In [ ]:
rsf = outerfolds['ERpHER2n_Combined'][0]['model']


In [ ]:
result = permutation_importance(rsf, X_test, y_test, n_repeats=15, random_state=42)

In [ ]:
from sklearn.inspection import permutation_importance
import pickle
import pandas as pd
import numpy as np
from datetime import datetime

def save_checkpoint(results, checkpoint_file='permutation_checkpoint.pkl'):
    """Save results after each fold."""
    with open(checkpoint_file, 'wb') as f:
        pickle.dump(results, f)
    print(f"Checkpoint saved to {checkpoint_file}")

def load_checkpoint(checkpoint_file='permutation_checkpoint.pkl'):
    """Load existing results if available."""
    try:
        with open(checkpoint_file, 'rb') as f:
            return pickle.load(f)
    except FileNotFoundError:
        return {}

# Main execution
results = load_checkpoint()  # Resume if interrupted

print(f"\n{'='*80}")
print(f"PERMUTATION IMPORTANCE ANALYSIS")
print(f"Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Configuration:")
print(f"  - Folds: 10")
print(f"  - Features per fold: ~5,000")
print(f"  - Trees per model: 800")
print(f"  - n_repeats: 3")
print(f"  - max_samples: 0.5")
print(f"Estimated total time: 8-12 hours")
print(f"{'='*80}\n")

for entry in ERpHER2n_Combined_OuterDicts:
    fold = entry["fold"]
    
    # Skip if already computed
    if fold in results:
        print(f"Fold {fold}: Already computed, skipping...")
        continue
    
    if entry["model"] is None:
        print(f"Fold {fold}: No model, skipping...")
        continue
    
    model = entry["model"]
    test_idx = entry["test_idx"]
    train_idx = entry["train_idx"]
    
    features_to_use = entry.get("features_after_filter2") or entry.get("features_after_filter1")
    X_test = X.iloc[test_idx][features_to_use]
    y_test = y[test_idx]
    
    print(f"\n{'─'*80}")
    print(f"   Fold {fold}/{len(ERpHER2n_Combined_OuterDicts)-1}")
    print(f"   Features: {len(features_to_use):,}")
    print(f"   Test samples: {len(X_test)}")
    print(f"   Started: {datetime.now().strftime('%H:%M:%S')}")
    
    # Estimate time
    est_time_min = (len(features_to_use) * 3 * 0.4) / 60  # Conservative estimate
    print(f"   Estimated duration: {est_time_min:.0f}-{est_time_min*1.5:.0f} minutes")
    print(f"{'─'*80}")
    
    try:
        # Compute permutation importance
        start_time = datetime.now()
        perm_result = permutation_importance(
            model, 
            X_test, 
            y_test, 
            n_repeats=3,
            random_state=42, 
            n_jobs=-1,
            max_samples=0.5
        )
        
        elapsed = (datetime.now() - start_time).total_seconds() / 60
        
        # Store results
        results[fold] = {
            'importances_mean': perm_result.importances_mean,
            'importances_std': perm_result.importances_std,
            'features': features_to_use,
            'elapsed_minutes': elapsed,
            'timestamp': datetime.now().isoformat()
        }
        
        # Save checkpoint after each fold
        save_checkpoint(results)
        
        print(f"   Completed in {elapsed:.1f} minutes")
        
        # Show top 5 features
        top_idx = np.argsort(perm_result.importances_mean)[-5:][::-1]
        print(f"   Top 5 features:")
        for idx in top_idx:
            print(f"      {features_to_use[idx]}: {perm_result.importances_mean[idx]:.4f}")
        
        # Progress update
        completed = len(results)
        remaining = 10 - completed
        avg_time = np.mean([results[f]['elapsed_minutes'] for f in results])
        est_remaining = remaining * avg_time
        
        print(f"\n   Progress: {completed}/10 folds complete")
        print(f"   Average time per fold: {avg_time:.1f} minutes")
        print(f"   Estimated time remaining: {est_remaining:.0f} minutes ({est_remaining/60:.1f} hours)")
        
    except Exception as e:
        print(f"   Error in fold {fold}: {e}")
        import traceback
        traceback.print_exc()
        continue

# Final save
print(f"\n{'='*80}")
print(f"ALL FOLDS COMPLETE!")
print(f"Finished: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"{'='*80}\n")

# Save final results
with open('permutation_importance_final.pkl', 'wb') as f:
    pickle.dump(results, f)

# Quick summary
print("Summary of computation times:")
for fold in sorted(results.keys()):
    print(f"  Fold {fold}: {results[fold]['elapsed_minutes']:.1f} minutes")

total_time = sum(results[fold]['elapsed_minutes'] for fold in results)
print(f"\nTotal computation time: {total_time:.1f} minutes ({total_time/60:.1f} hours)")